In [99]:
import requests
import urllib.request
import numpy as np
import json, random, copy, os
from tqdm import tqdm
import pickle

In [3]:
USER_AGENT_LIST = [
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
            "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
            "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
            "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
            "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
            "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
            "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
            "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 LBBROWSER",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; 360SE)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (iPad; U; CPU OS 4_2_1 like Mac OS X; zh-cn) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8C148 Safari/6533.18.5",
            "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0b13pre) Gecko/20110307 Firefox/4.0b13pre",
            "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:16.0) Gecko/20100101 Firefox/16.0",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
            "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
        ]

In [4]:
API_URL = "http://128.2.205.68:5000/metric_post"

In [14]:
def request(params):
  
    global USER_AGENT_LIST
    global API_URL

    params['format'] = 'json'
    if not 'action' in params:
        params['action'] = 'query'

    headers = {
        'User-Agent': random.choice(USER_AGENT_LIST)#'wikipedia (https://github.com/goldsmith/Wikipedia/)'
    }

    r = requests.post(API_URL, params=params, headers=headers)
    return r

In [35]:
c = ["welcome to visit paradise harbour"]
a = ["thanks for coming to paradise harbour",
    "welcome to paradise hotel",
    "welcome you to paradise harbour",
    ]
a2 = ["thanks for coming to paradise",
    "welcome you to paradise hotel",
    "welcome you to paradise resort",
    ]
ka = "paradise resort"
params = {'A': json.dumps([a, a2]), 'C': json.dumps([c, c]), 'Keyword': json.dumps([ka, ka]), 'Qcate': json.dumps(['text', 'text'])}
result = request(params)

In [36]:
print(result.json()['Fluency'])
print(result.json()['Accuracy'])
print(result.json()['mul'])

0.0753
0.5
0.0376


In [78]:
def eval_file(file, Qcate_breakdown=['all']):
    with open(os.path.join("/home/yingshac/CYS/WebQnA/VLP/vlp/light_output/", file), "r") as fp:
        lines = fp.readlines()
        header = lines[0].strip().split('\t')
        rows = lines[1:]
    key = dict(zip(header, range(len(header))))
    print(key)

    output_A = []
    output_C = []
    output_KA = []
    output_QC = []
    # Guid	Qcate	Q	A	Keywords_A	Output_conf	Output
    total = len(rows)
    count = 0
    fl = 0
    acc = 0
    mul = 0
    for r in tqdm(rows):

        datum = r.strip().split('\t')
        Qcate = datum[key['Qcate']]
        if (not 'all' in Qcate_breakdown) and (not Qcate in Qcate_breakdown): continue

        O = json.loads(datum[key['Output']])
        C = [O[0]]
        Keywords_A = datum[key['Keywords_A']]
        A = json.loads(datum[key['A']])

        output_A.append(A)
        output_C.append(C)
        output_KA.append(Keywords_A)
        output_QC.append(Qcate)
        
        if len(output_A) == 100:
            params = {'A': json.dumps(output_A), 'C': json.dumps(output_C), 'Keyword': json.dumps(output_KA), 'Qcate': json.dumps(output_QC)}
            result = request(params).json()
            fl += 100*result['Fluency']
            acc += 100*result['Accuracy']
            mul += 100*result['mul']
            output_A = []
            output_C = []
            output_KA = []
            output_QC = []
            count += 100
    if len(output_A) > 0:
        bs = len(output_A)
        params = {'A': json.dumps(output_A), 'C': json.dumps(output_C), 'Keyword': json.dumps(output_KA), 'Qcate': json.dumps(output_QC)}
        result = request(params).json()
        fl += bs*result['Fluency']
        acc += bs*result['Accuracy']
        mul += bs*result['mul']
        count += bs
    assert count == total
    result = {'Fluency': round(fl/total, 4), 'Accuracy': round(acc/total, 4), 'mul': round(mul/total, 4)}
    return(result)

In [55]:
A = ["Hi", 'Hello']
B = ["Hi", 'Hello', "Hey"]
output_A = [A, B]
json.loads(json.dumps(output_A))

[['Hi', 'Hello'], ['Hi', 'Hello', 'Hey']]

In [79]:
file = "detectron_both_qa_upd/qa_infr/test_qainfr_no_eval_-1_beam5_txt_True_step11_txt_dataset_0823_clean_te.tsv"
eval_file(file)

  0%|          | 0/4076 [00:00<?, ?it/s]

{'Guid': 0, 'Qcate': 1, 'Q': 2, 'A': 3, 'Keywords_A': 4, 'Output_conf': 5, 'Output': 6}


100%|██████████| 4076/4076 [05:42<00:00, 11.89it/s]


{'Accuracy': 0.5037, 'Fluency': 0.3301, 'mul': 0.2762}

In [100]:
Guid_map_0904 = pickle.load(open("Guid_map_0904.pkl", "rb"))
print(len(set(Guid_map_0904.keys())))
print(len(set(Guid_map_0904.values())))
image_id_map_0904 = pickle.load(open("image_id_map_0904.pkl", "rb"))
print(len(image_id_map_0904))

49272
49272
389750


In [111]:
list(image_id_map_0904.keys())[:10]

['20031027',
 '10026934',
 '20036023',
 '10136564',
 '10065736',
 '20014244',
 '10322283',
 '10334570',
 '10282838',
 '20142696']

In [101]:
### Generate prediction file in EvalAI format for testing
with open("/home/yingshac/CYS/WebQnA/VLP/vlp/light_output/detectron_both_qa_sentence/qa_infr/test_qainfr_no_eval_-1_beam5_img_True_True_step12_pred_dataset_th25_test_-1_step4_img_16_True_True_img_dataset_0823_clean_te_UNknown_modality.tsv", "r") as fp:
    lines = fp.readlines()
    header = lines[0].strip().split('\t')
    rows = lines[1:]
pred_dataset = json.load(open("/home/yingshac/CYS/WebQnA/VLP/vlp/light_output/filter_both_x_detectron/pred_dataset_th25_test_-1_step4_img_16_True_True_img_dataset_0823_clean_te_UNknown_modality.json", "r"))
print(len(pred_dataset), len(rows))
key = dict(zip(header, range(len(header))))
print(key)


3464 3464
{'Guid': 0, 'Qcate': 1, 'Q': 2, 'A': 3, 'Keywords_A': 4, 'Output_conf': 5, 'Output': 6}


In [115]:
Guid2srcPred = {}
for k in pred_dataset:
    Guid = pred_dataset[k]['Guid']
    Guid2srcPred[Guid] = []
    for i in pred_dataset[k]['txt_posFacts']:
        Guid2srcPred[Guid].append(i['fact'])
    for i in pred_dataset[k]['img_posFacts']:
        imid = str(i['image_id'])
        if len(imid) < 8:
            imid = "0"*(8-len(imid)) + imid
        try: Guid2srcPred[Guid].append(image_id_map_0904[imid])
        except: pass
print(len(Guid2srcPred))

3464


In [116]:
user_annotation = {}
for r in rows:

    datum = r.strip().split('\t')
    Guid = datum[key['Guid']]
    
    O = json.loads(datum[key['Output']])        
    
    user_annotation[Guid_map_0904[Guid]] = {
        'sources': Guid2srcPred[Guid],
        'answer': O[0]
    }
json.dump(user_annotation, open("/home/yingshac/CYS/WebQnA/VLP/vlp/tmp/evalai_submission_test2.json", "w"), indent=4)

In [84]:
import sys
sys.path.append("/home/yingshac/CYS/WebQnA/EvalAI-WebQA/evaluation_script/")
from main import evaluate

In [118]:
evaluate("", "/home/yingshac/CYS/WebQnA/VLP/vlp/tmp/evalai_submission_test2.json", "test")

Starting Evaluation.....
Evaluating for WebQA testing split
Found 3464 predictions.


Evaluated 3464 predictions.

Completed evaluation for WebQA testing split


{'result': [{'test_split': {'QA': 0.1628,
    'QA-Acc': 0.25,
    'QA-FL': 0.4987,
    'Retrieval': 0.6731}}],
 'submission_result': {'QA': 0.1628,
  'QA-Acc': 0.25,
  'QA-FL': 0.4987,
  'Retrieval': 0.6731}}